In [2]:
from torch.distributions import Categorical
import gym
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim

In [3]:
gamma = 0.99
episodes = 500

In [4]:
class Pi(nn.Module):
    # Pi constructs the policy network that is a simple one-layer MLP with 64 hidden units 
    def __init__(self, in_dim, out_dim):
        super(Pi, self).__init__()
        layers = [
            nn.Linear(in_dim, 64),
            nn.ReLU(),
            nn.Linear(64, out_dim)
        ]
        self.model = nn.Sequential(*layers)
        self.onpolicy_reset()
        self.train()
    def onpolicy_reset(self):
        self.log_probs = []
        self.rewards = []
        
    def default(state, algorithm, body):
        pdparam = calc_pdparam(state, algorithm, body)
        action = sample_action(body.ActionPD, pdparam)
        return action
        
    def epsilon_greedy(state, algorithm, body):
        epsilon = body.explore_var
        if epsilon > np.random.rand():
            return random(state, algorithm, body)
        else:
            return default(state, algorithm, body)

    def forward(self, x):
        pdparam = self.model(x)
        return pdparam

    # act defines the method to produce action
    def act(self, state):
        x = torch.from_numpy(state.astype(np.float32)) # to tensor
        pdparam = self.forward(x) # forward pass
        pd = Categorical(logits = pdparam) #probability distribution
        action = pd.sample() # pi (a|s) in action via pd
        log_prob = pd.log_prob(action) #log_prob of pi(a|s)
        self.log_probs.append(log_prob) #store for training
        return action.item()
    
    def calc_q_loss(self, batch):
        states = batch['states']
        next_states = batch['next states']
        

In [5]:
def train(pi, optimizer):
    # Inner gradient-ascent loop of REINFORCE algorithm
    T = len(pi.rewards)
    rets = np.empty(T, dtype = np.float32) # the returns
    future_ret = 0.0
    # compute the returns efficiently
    for t in reversed(range(T)):
        future_ret = pi.rewards[t] + gamma * future_ret
        rets[t] = future_ret
    rets = torch.tensor(rets)
    log_probs = torch.stack(pi.log_probs)
    loss = -log_probs * rets # gradient term; NEGATIVE for maximizing
    loss = torch.sum(loss)
    optimizer.zero_grad()
    loss.backward() # backpropagate, compute gradients
    optimizer.step() # gradient-ascent, update the weights
    return loss

In [6]:
def main():
    env = gym.make('CartPole-v0')
    in_dim = env.observation_space.shape[0] # 4
    out_dim = env.action_space.n # 2
    pi = Pi(in_dim, out_dim) # policy pi_theta for REINFORCE
    optimizer = optim.Adam(pi.parameters(), lr = 0.01)
    for epi in range(episodes):
        state = env.reset()
        for t in range(200): # cartpole max timestep is 200
            action = pi.act(state)
            state, reward, done, _ = env.step(action)
            pi.rewards.append(reward)
            env.render()
            if done:
                break
        loss = train(pi, optimizer) # train per episode
        total_reward = sum(pi.rewards)
        solved = total_reward > 195.0
        pi.onpolicy_reset() # onpolicy: clear memory after training
        print(f'Episode {epi}, loss: {loss}, \
        tota_reward: {total_reward}, solved: {solved}')

In [ ]:
if __name__ == '__main__':
    main()

Episode 0, loss: 122.9510498046875,         tota_reward: 19.0, solved: False
Episode 1, loss: 408.8636779785156,         tota_reward: 36.0, solved: False
Episode 2, loss: 99.8899154663086,         tota_reward: 16.0, solved: False
Episode 3, loss: 468.4656066894531,         tota_reward: 39.0, solved: False
Episode 4, loss: 187.10537719726562,         tota_reward: 24.0, solved: False
Episode 5, loss: 148.90402221679688,         tota_reward: 21.0, solved: False
Episode 6, loss: 184.5225067138672,         tota_reward: 24.0, solved: False
Episode 7, loss: 37.05106735229492,         tota_reward: 10.0, solved: False
Episode 8, loss: 50.24732971191406,         tota_reward: 12.0, solved: False
Episode 9, loss: 199.95860290527344,         tota_reward: 24.0, solved: False
Episode 10, loss: 184.0217742919922,         tota_reward: 24.0, solved: False
Episode 11, loss: 129.0937957763672,         tota_reward: 20.0, solved: False
Episode 12, loss: 334.2324523925781,         tota_reward: 33.0, solved: 